## Imports

In [ ]:
import copy
import logging
from pathlib import Path
from typing import Dict
import math

import hydra
import matplotlib
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import omegaconf
import seaborn as sns
import torch  # noqa
import wandb
from hydra.utils import instantiate
from matplotlib import tri
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from omegaconf import DictConfig
from pytorch_lightning import LightningModule
from scipy.stats import qmc
from torch.utils.data import DataLoader
from tqdm import tqdm
from ccmm.matching.utils import perm_indices_to_perm_matrix
from ccmm.utils.utils import normalize_unit_norm, project_onto
from ccmm.matching.frank_wolfe_sync_matching import frank_wolfe_synchronized_matching
from ccmm.matching.utils import perm_matrix_to_perm_indices

from nn_core.callbacks import NNTemplateCore
from nn_core.common import PROJECT_ROOT
from nn_core.common.utils import seed_index_everything
from nn_core.model_logging import NNLogger
from ccmm.utils.utils import fuse_batch_norm_into_conv
from torch.utils.data import DataLoader, Subset, SubsetRandomSampler

import ccmm  # noqa
from ccmm.matching.utils import (
    apply_permutation_to_statedict,
    get_all_symbols_combinations,
    plot_permutation_history_animation,
    restore_original_weights,
)
from ccmm.utils.utils import (
    load_model_from_info,
    map_model_seed_to_symbol,
    save_factored_permutations,
)

from ccmm.matching.utils import load_permutations

from ccmm.utils.utils import vector_to_state_dict
import pytorch_lightning

In [ ]:
matplotlib.rcParams["font.family"] = "serif"
sns.set_context("talk")
matplotlib.rcParams["text.usetex"] = True
cmap_name = "coolwarm_r"

logging.getLogger("lightning.pytorch").setLevel(logging.WARNING)
logging.getLogger("torch").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.WARNING)
pylogger = logging.getLogger(__name__)

In [ ]:
from typing import List


def average_models(model_params, reduction_fn, coeffs=None):
    if not isinstance(model_params, List):
        model_params = list(model_params.values())

    if not coeffs:
        coeffs = [1] * len(model_params)

    return {k: reduction_fn(torch.stack([p[k] for p in model_params])) for k in model_params[0].keys()}


def trimmed_mean(tensors, trim_ratio=0.1):
    num_values = tensors.size(0)
    num_to_trim = int(trim_ratio * num_values)
    sorted_tensors = tensors.sort(dim=0).values
    trimmed_tensors = sorted_tensors[num_to_trim : num_values - num_to_trim]
    return trimmed_tensors.mean(dim=0)


def winsorize(tensor, limits=[0.2, 0.8]):
    lower, upper = torch.quantile(tensor, torch.tensor(limits).float(), dim=0)
    clipped = torch.clamp(tensor, min=lower, max=upper)
    return clipped.mean(dim=0)


def robust_mean(tensor, threshold=3.5):
    median_val = tensor.median(dim=0).values
    mad_val = (tensor - median_val).abs().median(dim=0).values
    mad_val[mad_val == 0] = 1  # Prevent division by zero
    z_score = 0.6745 * (tensor - median_val) / mad_val
    mask = (z_score.abs() < threshold).float()  # Create a mask to zero-out outliers
    filtered_tensor = tensor * mask  # Apply mask
    robust_mean_val = filtered_tensor.sum(dim=0) / mask.sum(dim=0)  # Compute mean only over non-outlier values
    return robust_mean_val

## Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

import hydra
from hydra import initialize, compose
from typing import Dict, List

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path=str("../conf"), job_name="merge_n_models")

In [ ]:
cfg = compose(config_name="merge_n_models", overrides=[])

In [ ]:
core_cfg = cfg  # NOQA
cfg = cfg.matching

seed_index_everything(cfg)

## Hyperparameters

In [ ]:
num_test_samples = 5000
num_train_samples = 5000

## Load dataset

In [ ]:
transform = instantiate(core_cfg.dataset.test.transform)

train_dataset = instantiate(core_cfg.dataset.train, transform=transform)
test_dataset = instantiate(core_cfg.dataset.test, transform=transform)

train_subset = Subset(train_dataset, list(range(num_train_samples)))
train_loader = DataLoader(train_subset, batch_size=5000, num_workers=cfg.num_workers)

test_subset = Subset(test_dataset, list(range(num_test_samples)))

test_loader = DataLoader(test_subset, batch_size=1000, num_workers=cfg.num_workers)

In [ ]:
trainer = instantiate(cfg.trainer, enable_progress_bar=False, enable_model_summary=False)

## Load models

In [ ]:
from ccmm.utils.utils import load_model_from_artifact

run = wandb.init(project=core_cfg.core.project_name, entity=core_cfg.core.entity, job_type="matching")

# {a: 1, b: 2, c: 3, ..}
symbols_to_seed: Dict[int, str] = {map_model_seed_to_symbol(seed): seed for seed in cfg.model_seeds}

artifact_path = (
    lambda seed: f"{core_cfg.core.entity}/{core_cfg.core.project_name}/{core_cfg.dataset.name}_{core_cfg.model.model_identifier}_{seed}:v0"
)

# {a: model_a, b: model_b, c: model_c, ..}
models: Dict[str, LightningModule] = {
    map_model_seed_to_symbol(seed): load_model_from_artifact(run, artifact_path(seed)) for seed in cfg.model_seeds
}

num_models = len(models)

pylogger.info(f"Using {num_models} models with architecture {core_cfg.model.model_identifier}")

In [ ]:
# always permute the model having larger character order, i.e. c -> b, b -> a and so on ...
symbols = set(symbols_to_seed.keys())
symbols = symbols.difference({"o"})  # "o" is the model trained over all the dataset

sorted_symbols = sorted(symbols, reverse=False)

# (a, b), (a, c), (b, c), ...
all_combinations = get_all_symbols_combinations(symbols)
# combinations of the form (a, b), (a, c), (b, c), .. and not (b, a), (c, a) etc
canonical_combinations = [(source, target) for (source, target) in all_combinations if source < target]

## Matching

In [ ]:
pylogger.info(f"Matching the following model pairs: {canonical_combinations}")

### Load permutation specification

In [ ]:
permutation_spec_builder = instantiate(core_cfg.model.permutation_spec_builder)
permutation_spec = permutation_spec_builder.create_permutation()

ref_model = list(models.values())[0]
assert set(permutation_spec.layer_and_axes_to_perm.keys()) == set(ref_model.model.state_dict().keys())

## Merging

In [ ]:
model_merger = instantiate(cfg.merger, permutation_spec=permutation_spec)

pylogger.info(f"Merger: {model_merger.name}")

In [ ]:
max_iter = 200
initialization_method = "identity"
keep_soft_perms = True

symbols = list(models.keys())

merged_model = copy.deepcopy(models[symbols[0]])

perm_indices, opt_infos = frank_wolfe_synchronized_matching(
    models=models,
    perm_spec=permutation_spec,
    symbols=symbols,
    combinations=canonical_combinations,
    max_iter=max_iter,
    initialization_method=initialization_method,
    keep_soft_perms=keep_soft_perms,
    verbose=False,
)

In [ ]:
# perm_symbols = list(perm_indices['a'].keys())
# # perm_symbols = ['P_bg0', 'P_blockgroup3.block2_inner']
# k = 0
# K = 5
# for perm_symb in perm_symbols:
#     perms = {symb: [perm[symb][perm_symb] for perm in opt_infos['perm_history'][k:K]] for symb in symbols }

#     fig, ax = plt.subplots(5, K-k, figsize=(20, 20))

#     for i in range(K-k):
#         for j, symb in enumerate(symbols):
#             ax[j, i].imshow(perms[symb][i].cpu(), cmap='gray')
#             ax[j, i].axis('off')
#             ax[j, i].set_title(symb)
#     plt.show()

In [ ]:
perm_symbols = list(perm_indices["a"].keys())
print(perm_symbols)

In [ ]:
plt.plot(opt_infos["obj_values"])

In [ ]:
plt.plot([step_size for ind, step_size in enumerate(opt_infos["step_sizes"]) if ind % 2 == 1])

In [ ]:
plt.plot([step_size for ind, step_size in enumerate(opt_infos["step_sizes"]) if ind % 2 == 0])

In [ ]:
from ccmm.matching.weight_matching import solve_linear_assignment_problem

hard_perms = {
    symb: {p: solve_linear_assignment_problem(perm) for p, perm in perm_indices[symb].items()} for symb in symbols
}

soft_perms = copy.deepcopy(perm_indices)

In [ ]:
plot_perms = False
if plot_perms:
    perm_symbols = list(perm_indices["a"].keys())

    for perm_symbol in perm_symbols:
        fig, axs = plt.subplots(2, len(symbols), figsize=(40, 20))
        fig.suptitle(f"Permutation: {perm_symbol}", fontsize=16)
        for i, symbol in enumerate(symbols):

            ax0 = axs[0][i]
            ax0.set_title(symbol)
            ax0.imshow(soft_perms[symbol][perm_symbol].cpu(), cmap=cmap_name)
            ax0.colorbar = plt.colorbar(
                matplotlib.cm.ScalarMappable(norm=colors.Normalize(vmin=0, vmax=1), cmap=cmap_name), ax=ax0
            )

            ax1 = axs[1][i]
            ax1.set_title(symbol)
            ax1.imshow(perm_indices_to_perm_matrix(hard_perms[symbol][perm_symbol]), cmap=cmap_name)
            ax1.colorbar = plt.colorbar(
                matplotlib.cm.ScalarMappable(norm=colors.Normalize(vmin=0, vmax=1), cmap=cmap_name), ax=ax1
            )

In [ ]:
perm_name = "P_blockgroup2.block3_inner"
nonzero_idxs = (soft_perms["a"][perm_name] > 0) & (soft_perms["a"][perm_name] < 1)

soft_perms["a"][perm_name][nonzero_idxs]

In [ ]:
def get_models_permuted_to_univ(perms, models, symbols, keep_soft_perms=False):
    models_permuted_to_universe = {symbol: copy.deepcopy(model) for symbol, model in models.items()}

    for symbol in symbols:
        perms_to_apply = {}

        for perm_name in perms[symbol].keys():
            perm = perms[symbol][perm_name]

            if keep_soft_perms:
                perm = perm.T
                perm_to_apply = perm
            else:
                perm = perm_indices_to_perm_matrix(perm).T
                perm_to_apply = perm_matrix_to_perm_indices(perm)

            perms_to_apply[perm_name] = perm_to_apply

        updated_params = apply_permutation_to_statedict(
            permutation_spec, perms_to_apply, models[symbol].model.state_dict()
        )
        models_permuted_to_universe[symbol].model.load_state_dict(updated_params)

    return models_permuted_to_universe


models_permuted_to_universe = get_models_permuted_to_univ(soft_perms, models, symbols, keep_soft_perms=True)

In [ ]:
from ccmm.matching.utils import unfactor_permutations

models_permuted_pairwise = {
    symbol: {other_symb: None for other_symb in set(symbols).difference(symbol)} for symbol in symbols
}
pairwise_permutations = unfactor_permutations(hard_perms)

for fixed, permutee in all_combinations:
    ref_model = copy.deepcopy(models["a"])

    permuted_params = apply_permutation_to_statedict(
        permutation_spec, pairwise_permutations[fixed][permutee], models[permutee].model.state_dict()
    )

    ref_model.model.load_state_dict(permuted_params)
    models_permuted_pairwise[fixed][permutee] = ref_model

In [ ]:
models = {symb: model.to("cpu") for symb, model in models.items()}

#### Check if they are orthogonal

In [ ]:
flat_models = {symbol: torch.nn.utils.parameters_to_vector(model.parameters()) for symbol, model in models.items()}
flat_models_permuted_to_universe = {
    symbol: torch.nn.utils.parameters_to_vector(model.parameters())
    for symbol, model in models_permuted_to_universe.items()
}

In [ ]:
def plot_cosine_similarities(models):
    # matrix of the cosine products
    cosine_matrix = np.zeros((len(models), len(models)))

    for i, (symbol_i, model_i) in enumerate(models.items()):
        for j, (symbol_j, model_j) in enumerate(models.items()):
            cosine_matrix[i, j] = (model_i @ model_j) / (torch.norm(model_i) * torch.norm(model_j))

    plt.figure(figsize=(5, 5))
    sns.heatmap(cosine_matrix, annot=True, cmap="viridis")
    plt.title("Cosine Similarity Matrix")
    plt.ylabel("Model Symbol")
    plt.show()

In [ ]:
plot_cosine_similarities(flat_models)
plot_cosine_similarities(flat_models_permuted_to_universe)

### Try fancy averaging functions

In [ ]:
from functools import partial


model_params = [model.model.state_dict() for model in models_permuted_to_universe.values()]

red_fns = {"mean": partial(torch.mean, dim=0), "trimmed": trimmed_mean, "winsor": winsorize, "filter": robust_mean}
merged_params = average_models(model_params, reduction_fn=red_fns["mean"])
merged_model.model.load_state_dict(merged_params)

### Evaluate

In [ ]:
loader = test_loader
trainer.test(merged_model, loader)

#### Repair

In [ ]:
from ccmm.matching.repair import repair_model

repaired_model = repair_model(merged_model, models_permuted_to_universe, train_loader)

In [ ]:
trainer.test(repaired_model, loader)

## Fisher stuff

In [ ]:
models_permuted_to_universe = {symb: model.to("cpu") for symb, model in models_permuted_to_universe.items()}
fishers = {}

In [ ]:
from ccmm.matching.fisher_merging import compute_fisher_for_model

num_fisher_samples = 2000
fisher_train_subset = Subset(train_dataset, list(range(num_fisher_samples)))

fisher_train_loader = DataLoader(fisher_train_subset, batch_size=8)

num_classes = core_cfg.dataset.num_classes

fishers = {
    symbol: compute_fisher_for_model(model.model, fisher_train_loader, num_classes)
    for symbol, model in models_permuted_to_universe.items()
}

In [ ]:
for model in models_permuted_to_universe.values():
    model.to("cpu")

In [ ]:
merged_model = copy.deepcopy(models["a"])
for param_name, param in merged_model.model.named_parameters():
    param.data.zero_()

In [ ]:
target_model = copy.deepcopy(models_permuted_to_universe["a"])

In [ ]:
for symbol, model in models_permuted_to_universe.items():
    for param_name, param in model.model.named_parameters():

        to_add = torch.zeros_like(param.data)
        target_param = target_model.model.state_dict()[param_name].data

        fish_coeff = fishers[symbol][param_name]

        all_fishers = torch.stack([fishers[symb][param_name] for symb in symbols])
        fish_coeff = fish_coeff / all_fishers.sum(dim=0)

        tol = 1e-12

        num_small_fish = (fish_coeff < tol).sum()
        ratio_small_fish = num_small_fish / fish_coeff.numel()

        pylogger.info(f"Number of small fish: {num_small_fish}, ratio: {ratio_small_fish}")
        pylogger.info(f"Average fisher: {fish_coeff.mean()}")

        to_add[fish_coeff < tol] = target_param[fish_coeff < tol] * (1 / num_models)

        to_add[fish_coeff >= tol] = param.data[fish_coeff >= tol] * (1 / num_models) * fish_coeff[fish_coeff >= tol]

        merged_model.model.state_dict()[param_name].add_(to_add)

In [ ]:
trainer.test(merged_model, loader)

In [ ]:
trainer.test(target_model, loader)

In [ ]:
repaired_model = repair_model(merged_model, models_permuted_to_universe, train_loader)

In [ ]:
trainer.test(repaired_model, loader)

## Try using one of the models as reference model

In [ ]:
results = {symbol: {"vanilla": None, "merged": None} for symbol in symbols}

for symbol in symbols:

    mapped_models = {
        other_symb: models_permuted_pairwise[symbol][other_symb]
        for other_symb, model in models.items()
        if other_symb != symbol
    }
    mapped_params = {symb: model.model.state_dict() for symb, model in mapped_models.items()}

    merged_model = copy.deepcopy(models[symbol])

    mean_model = average_models(mapped_params, reduction_fn=red_fns["mean"])

    merged_model.model.load_state_dict(mean_model)

    vanilla_res = trainer.test(merged_model, loader)[0]

    repaired_model = repair_model(merged_model, mapped_models, train_loader)

    repair_res = trainer.test(repaired_model, loader)[0]

    results[symbol]["vanilla"] = vanilla_res

## Try subsets

In [ ]:
symbol_subsets = {symbol: set(symbols).difference(symbol) for symbol in symbols}

merged_model = copy.deepcopy(models[symbols[0]])

results = {}
merged_models = {}

for symbol_subset in symbol_subsets.values():

    combinations = get_all_symbols_combinations(symbol_subset)
    canonical_combinations = [(source, target) for (source, target) in combinations if source < target]
    model_subset = {symb: models[symb] for symb in symbol_subset}

    perm_indices, _ = frank_wolfe_synchronized_matching(
        models=model_subset,
        perm_spec=permutation_spec,
        symbols=list(symbol_subset),
        combinations=canonical_combinations,
        max_iter=max_iter,
        initialization_method=initialization_method,
        keep_soft_perms=keep_soft_perms,
    )

    pylogger.info(f"Symbol subset: {symbol_subset}")

    models_to_univ_subset = get_models_permuted_to_univ(model_subset, symbol_subset, keep_soft_perms)

    model_params = [model.model.state_dict() for model in models_to_univ_subset.values()]

    merged_params = average_models(model_params, reduction_fn=red_fns["mean"])
    merged_model.model.load_state_dict(merged_params)

    merged_results = trainer.test(merged_model, loader)[0]

    repaired_model = repair_model(merged_model, models_to_univ_subset, train_loader)

    repair_results = trainer.test(repaired_model, loader)[0]

    results[tuple(symbol_subset)] = {"merged": merged_results, "repaired": repair_results}
    merged_models[tuple(symbol_subset)] = {"merged": merged_model, "repaired": repaired_model}

## Try subsets without re-aligning

In [ ]:
all_combinations = get_all_symbols_combinations(symbols)
canonical_combinations = [(source, target) for (source, target) in all_combinations if source < target]

perm_indices, _ = frank_wolfe_synchronized_matching(
    models=models,
    perm_spec=permutation_spec,
    symbols=symbols,
    combinations=canonical_combinations,
    max_iter=max_iter,
    initialization_method=initialization_method,
    keep_soft_perms=keep_soft_perms,
)

In [ ]:
merged_model = copy.deepcopy(models[symbols[0]])
models_permuted_to_universe = get_models_permuted_to_univ(models, symbols, keep_soft_perms)

results_norealign = {}
merged_models_norealign = {}

for symbol_subset in symbol_subsets.values():
    pylogger.info(f"Symbol subset: {symbol_subset}")

    models_to_univ_subset = {symb: models_permuted_to_universe[symb] for symb in symbol_subset}

    model_params = [model.model.state_dict() for model in models_to_univ_subset.values()]

    merged_params = average_models(model_params, reduction_fn=red_fns["mean"])
    merged_model.model.load_state_dict(merged_params)

    merged_results = trainer.test(merged_model, loader)[0]

    repaired_model = repair_model(merged_model, models_to_univ_subset, train_loader)

    repair_results = trainer.test(repaired_model, loader)[0]

    results_norealign[tuple(symbol_subset)] = {"merged": merged_results, "repaired": repair_results}
    merged_models_norealign[tuple(symbol_subset)] = {"merged": merged_model, "repaired": repaired_model}

## Analyze models as vectors

### Flatten models

In [ ]:
flat_models_permuted_pairwise = {
    symbol: {
        other_symb: torch.nn.utils.parameters_to_vector(model.parameters()) for other_symb, model in models.items()
    }
    for symbol, models in models_permuted_pairwise.items()
}

## Interpolation curves

In [ ]:
def linear_interpolation(model_a, model_b, lamb):
    return (1 - lamb) * model_a + lamb * model_b


def get_interp_curve(lambdas, model_a, model_b, ref_model, test_loader):
    interp_losses = []
    interp_accs = []

    for lamb in lambdas:
        interp_params = linear_interpolation(model_a=model_a, model_b=model_b, lamb=lamb)

        interp_params = vector_to_state_dict(interp_params, ref_model.model)

        ref_model.model.load_state_dict(interp_params)
        results = trainer.test(ref_model, test_loader, verbose=False)

        interp_losses.append(results[0][f"loss/test"])
        interp_accs.append(results[0][f"acc/test"])

    return interp_losses, interp_accs

## Average in the universe

In [ ]:
merged_model = copy.deepcopy(models["a"])

vec = torch.nn.utils.parameters_to_vector(merged_model.parameters())

vec = torch.stack([model for model in flat_models_permuted_to_universe.values()]).mean(dim=0)

torch.nn.utils.vector_to_parameters(vec, merged_model.parameters())

### Plot LMC

In [ ]:
def plot_lmc(values, lambdas, labels, axis=None):

    num_curves = len(values)
    transparencies = np.linspace(0.5, 1, num_curves)
    linewidths = np.linspace(2.0, 4.0, num_curves)

    for i, (val, label) in enumerate(zip(values, labels)):
        if axis is None:
            axis = plt

        axis.plot(lambdas, val, label=label, alpha=transparencies[i], linewidth=linewidths[i])

    plt.legend()

In [ ]:
p_perm_to_a = models_permuted_pairwise["a"]["b"]
lambdas = np.linspace(0, 1, 3)

from ccmm.utils.utils import get_interpolated_loss_acc_curves

loss, acc = get_interpolated_loss_acc_curves(
    model_a=models["a"],
    model_b=p_perm_to_a,
    lambdas=lambdas,
    ref_model=ref_model,
    trainer=trainer,
    loader=test_dataloaders[0],
)

In [ ]:
values = [loss]
labels = ["Loss"]
plot_lmc(values, lambdas, labels)

In [ ]:
values = [acc]
labels = ["Acc"]
plot_lmc(values, lambdas, labels)